## Projeto de estabilização de video

Projeto em que, dado o conhecimento adquirido sobre fluxo ótico de imagens, criar um estabilizador de webcam compensando deslocamentos da camera ou do agente que a camera esta capturando


In [1]:
%reset -f
%matplotlib inline

#!pip install opencv-python

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv.__version__)

3.4.2


### Primeira versão: cálculo do deslocamento com optical flow de determinados pontos da imagem

A primeira versão do código utiliza-se de um selecionador de boas features para serem rastreadas na imagem, calculando o fluxo ótico baseado no movimento delas ao longo da imagem

In [7]:
# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))


captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

previous = captura.read()[1]
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

media = np.array([0.0,0.0])

while(1):
    ret, frame = captura.read()
    
    actual_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    delta = good_new - good_old
    media -= np.mean(delta, axis=0)
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    
    img = frame
    
    img_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rows, cols = img_gray.shape
    
    M = np.array([[1, 0, media[0]], [0, 1, media[1]]], dtype=np.float32)
    img_shifted = cv.warpAffine(img, M, (cols,rows))  # Terceiro argumento é o tamanho da imagem resultante.
    
    cv.imshow("Video", img_shifted)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)

    # Pressione ESC para sair do loop
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    
captura.release()
cv.destroyAllWindows()

O problema desta primeira versão é que o uso de pontos pre determinados da imagem para calcular o fluxo optico da imagem pode ser uma ma escolha, uma vez que depende-se do algoritmo para fazer a escolha dos pontos. Isto pode resultar em escolha de features que simplesmente não estão em movimento, prejudicando o calculo do fluxo otico medio da imagem, assim como o deslocamento de pontos que inevitavelmente vão sair do frame, perdendo a informação destes fluxos ópticos. Ainda ha erros associados ao algoritmo de captura de features, uma vez que alguns pontos de features que ja sairam do frame permanecem calculados como se estivessem ainda na borda da mesma.

### Segunda versão: calculo do dense optical flow de uma janela da imagem

A segunda versão utiliza-se do calculo do fluxo otico geral da tela. Ao inves de depender de algumas features rastreaveis, o metodo de farneback calcula o fluxo otico de todos os pontos do frame, e para cada pixel, frame a frame, determina-se um vetor com direção e magnitude na direção do movimento observado entre frames. O algoritmo abaixo utiliza-se do calculo do fluxo otico denso de uma janela central 

In [71]:
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

previous = captura.read()[1]
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)

media = np.array([0.0,0.0])

win_size = 120

while(1):
    ret, frame = captura.read()
    
    actual_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    
    
    # Calcula o Fluxo Otico
    flow = cv.calcOpticalFlowFarneback(previous_gray[int((previous_gray.shape[0]/2)-win_size):int((previous_gray.shape[0]/2)+win_size)][int((previous_gray.shape[1]/2)-win_size):int((previous_gray.shape[1]/2)+win_size)], 
                                       actual_gray[int((previous_gray.shape[0]/2)-win_size):int((previous_gray.shape[0]/2)+win_size)][int((previous_gray.shape[1]/2)-win_size):int((previous_gray.shape[1]/2)+win_size)], 
                                       None, 0.5, 3, 5, 3, 5, 1.2, 0)
    media_x = np.mean(flow[...,0])
    media_y = np.mean(flow[...,1])
    media -= [media_x, media_y]

    img = frame
    
    img_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rows, cols = img_gray.shape
    
    M = np.array([[1, 0, media[0]], [0, 1, media[1]]], dtype=np.float32)
    img_shifted = cv.warpAffine(img, M, (cols,rows))  # Terceiro argumento é o tamanho da imagem resultante.
    
    cv.imshow("Video", img_shifted)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()


    # Pressione ESC para sair do loop
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    
captura.release()
cv.destroyAllWindows()

### Otimizando a estabilizacao cortando as bordas:

Uma implementação que visa contornar o aparecimento das bordas pretas foi desenvolvido abaixo. O algoritmo tenta compensaras bordas pretas (e a consequente perda de proporcao da imagem) cortando a imagem no outro eixo tal que a mesma mantenha a proporção, mas cause um efeito de zoom.

In [80]:
captura = cv.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv.CAP_PROP_BUFFERSIZE, 1)

previous = captura.read()[1]
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)

media = np.array([0.0,0.0])

win_size = 120

while(1):
    ret, frame = captura.read()
    
    actual_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    
    
    
    # Calcula o Fluxo Otico
    flow = cv.calcOpticalFlowFarneback(previous_gray[int((previous_gray.shape[0]/2)-win_size):int((previous_gray.shape[0]/2)+win_size)][int((previous_gray.shape[1]/2)-win_size):int((previous_gray.shape[1]/2)+win_size)], 
                                       actual_gray[int((previous_gray.shape[0]/2)-win_size):int((previous_gray.shape[0]/2)+win_size)][int((previous_gray.shape[1]/2)-win_size):int((previous_gray.shape[1]/2)+win_size)], 
                                       None, 0.5, 3, 5, 3, 5, 1.2, 0)
    media_x = np.mean(flow[...,0])
    media_y = np.mean(flow[...,1])
    media -= [media_x, media_y]

    img = frame
    
    
    
    if media[0] > 0:
        img = img[:,0:img.shape[1]-int(media[0])]
        
    else:
        img = img[:,-int(media[0]):img.shape[1]]
        
    if media[1] > 0:
        img = img[0:img.shape[0]-int(media[1]),:]
        
    else:
        img = img[-int(media[1]):img.shape[0],:]
    
    cut_0 = img.shape[0] - (3/4)*img.shape[1]
    cut_1 = img.shape[1] - (4/3)*img.shape[0]
    
    if cut_0 > 0:
        img = img[int(cut_0/2):img.shape[0]-int(cut_0/2),:]
        
    if cut_1 > 0:
        img = img[:,int(cut_1/2):img.shape[0]-int(cut_1/2)]
    
    
    img_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rows, cols = img_gray.shape
    
    cv.imshow("Video", img)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()

    # Pressione ESC para sair do loop
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

    
captura.release()
cv.destroyAllWindows()